In [1]:
import pandas as pd
import numpy as np
import requests
from html.parser import HTMLParser

### add `fork` column to repo with go.mod downloaded

In [3]:
df1 = pd.read_excel("repo-candidates/combined.xlsx")
df1.to_csv("repo-candidates/slim.csv", columns=['id', 'full_name', 'fork', 'stargazers_count', 'forks_count', 'created_at'])

## convert public name to github name

In [2]:
    
    
    
def convert_name_debug(public_name):
    request_url = f"https://{public_name}?go-get=1"
    resp = requests.get(request_url)
    return resp

    
def convert_name(public_name, parser):
    request_url = f"https://{public_name}"
    resp = requests.get(request_url)
    parser.feed(resp.text)
    return parser.github_name

class GoImportMetaHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._github_name = ""
       
    @property
    def github_name(self):
        return self._github_name
         
    def handle_starttag(self, tag, attrs):
        self._gith_name = ""
        if tag == "meta":
            imports_found = [attr for attr in attrs if attr[0] == "name" and attr[1] == "go-import"]
            contents = [attr[1] for attr in attrs if attr[0] == "content"]
            if imports_found and contents:
                comps = contents[0].split(' ')
                idx = comps[2].index("//")
                self._github_name = comps[2] if idx < 0 else comps[2][idx+2:]



In [5]:
resp = convert_name_debug("gopkg.in/juju/names.v2")
resp.text





'\n<html>\n<head>\n<meta name="go-import" content="gopkg.in/juju/names.v2 git https://gopkg.in/juju/names.v2">\n<meta name="go-source" content="gopkg.in/juju/names.v2 _ https://github.com/juju/names/tree/v2{/dir} https://github.com/juju/names/blob/v2{/dir}/{file}#L{line}">\n</head>\n<body>\ngo get gopkg.in/juju/names.v2\n</body>\n</html>\n'

In [6]:
parser = GoImportMetaHTMLParser()
parser.feed(resp.text)
parser.github_name

'gopkg.in/juju/names.v2'